# Implementation of a Spatiotemporal Self-Attention Based LSTNet

## Imports

In [1]:
import pandas as pd
import yfinance as yf
import tensorflow as tf
import numpy as np
import os
import csv
import keras_tuner
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold
from scikeras.wrappers import KerasRegressor
from data_processing import DataProcessor
from datetime import datetime
import matplotlib.pyplot as plt

#models
from models.simple_lstm import SimpleLSTM
from models.LSTNet import LSTNet
from models.simple_tcn import SimpleTCN
from models.mlp import MLP
from models.last_value_mlp import LastValueMLP
from models.attention_lstm import AttentionLSTM

Using TensorFlow backend


## Data Import and formatting

We are looking to work with multivariate time series data, which will be given to us as a 2D shape: # of timesteps x number of variables. We then need to format this data so that we split it into a train and test set, and use a sliding window approach to divide it into sequences. Then take the last x timesteps in every sequence and make those the labels. We will also of course normalize the data with z-score standardization.

First we will load data. Change the below code block to adjust for data source. Included in this code block should be any variable-specific feature extraction that needs to be down, such as calculating returns from price, etc

In [2]:
raw_data = pd.read_csv("DATA/nasdaq100/small/nasdaq100_padding.csv")

# #drop date
# raw_data.drop(["Date"], inplace=True, axis=1)

#set dataset name for logging
dataset_name = "nasdaq100"

#display info about loaded data
raw_data.info()
raw_data.dropna(inplace=True)

#we need to calculate returns, as this is more useful then the actual price. (price is a function of returns and prev price, so if we know returns we can always calculate price)
#raw_data = DataProcessor.calc_log_returns(raw_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40560 entries, 0 to 40559
Data columns (total 82 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAL     40560 non-null  float64
 1   AAPL    40560 non-null  float64
 2   ADBE    40560 non-null  float64
 3   ADI     40560 non-null  float64
 4   ADP     40560 non-null  float64
 5   ADSK    40560 non-null  float64
 6   AKAM    40560 non-null  float64
 7   ALXN    40560 non-null  float64
 8   AMAT    40560 non-null  float64
 9   AMGN    40560 non-null  float64
 10  AMZN    40560 non-null  float64
 11  ATVI    40560 non-null  float64
 12  AVGO    40560 non-null  float64
 13  BBBY    40560 non-null  float64
 14  BIDU    40560 non-null  float64
 15  BIIB    40560 non-null  float64
 16  CA      40560 non-null  float64
 17  CELG    40560 non-null  float64
 18  CERN    40560 non-null  float64
 19  CMCSA   40560 non-null  float64
 20  COST    40560 non-null  float64
 21  CSCO    40560 non-null  float64
 22

Then we do a train/test split, create sequences with sliding window, normalize data, and designate labels(what we are trying to predict). The length of each sequence is set by SEQ_LEN.

One question here is whether to normalize the target values. In other words, should we normalize data and then split the time series in to features and labels, or split first and then only normalize features. From past work, ~~I have seen better Mean Average Percent Error (MAPE) by doing the former, but intuition says it shouldn't matter.~~ From some reading, I have learned that normalizing all data, and not just the features, is a form of train-test contamination, as it allows some information from the test set to be encoded in the train set. Only features should be normalized. 

We also need to deal with missing values. For now, we will just drop the rows (timesteps) that have missing values but I think we should further consider the effects of this later down the line.

In [3]:
SEQ_LEN = 25

train_data, valtest_data = train_test_split(raw_data, test_size=0.3, shuffle=False) #it is imporant the data is not shuffled here, since this is time series data

train_data.dropna(inplace=True)
valtest_data.dropna(inplace=True)

val_data, test_data = train_test_split(valtest_data, test_size=0.5, shuffle=False)

train_sequences = DataProcessor.sliding_window_sequence(train_data, SEQ_LEN)
val_sequences = DataProcessor.sliding_window_sequence(val_data, SEQ_LEN)

train_x, train_y = DataProcessor.sequence_target_split(train_sequences, target_size=1)
val_x, val_y = DataProcessor.sequence_target_split(val_sequences, target_size=1)

train_x = DataProcessor.zscore_standardization(train_x)
val_x = DataProcessor.zscore_standardization(val_x)

We then need to convert this multiIndex DataFrame we have into a 3 dimensional numpy to feed into the model. Each of x_train, x_test, y_train, y_test will have the 3D shape num_sequences x timesteps_per_sequence x num_variables. For our y-values, we will only predict one timestep in the future for now, for simplicity's sake.

In [4]:
x_train = DataProcessor.fold_sequences(train_x)
x_val = DataProcessor.fold_sequences(val_x)
y_train = DataProcessor.fold_sequences(train_y)
y_val = DataProcessor.fold_sequences(val_y)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(28368, 24, 82)
(6060, 24, 82)
(28368, 82)
(6060, 82)


We also likely only want to predict one of the time series variables, so we can select which one we want that to be. In the code block below, choose the index of the variable to be selected.

In [5]:
Y_VAR = 0   #index of variable to be predicted

if len(y_train.shape) == 1:
    print("rrr")

    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
    y_train = y_train.reshape((y_train.shape[0], 1))
    y_val = y_val.reshape((y_val.shape[0], 1))

y_train = y_train[:,Y_VAR]
y_val = y_val[:,Y_VAR]

## Model Initialization

We create a short function to build the model, and set it up for a training run.

In [6]:
def build_model(hp):

    #change the model to run here
    model = AttentionLSTM(hp)
    model.build(x_train.shape)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.mean_absolute_error] 
    )

    return model

## Hyperparameter Optimization

Optionally, We will run a hyperparameter optimization on the model to find the best hyperparameters.

In [7]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_mean_absolute_error",
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="HP_OPTIMIZATION_LOGS"
)

hparam_optimize = False #change to false if want to use default hp

if hparam_optimize:

    print("Running hyperparameter optimization")
    tuner.search_space_summary()

    tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

    hparams = tuner.get_best_hyperparameters(1)[0]

else:

    hparams = keras_tuner.HyperParameters()

## Training

Now that we have our data properly formatted and hyperparameters decided, we will reinitialize the model with the chosen hyperparameters, train it, and test it. Please change the name of the model to be appropriate, as this name is used as a filename for saving the model.

In [8]:
model = build_model(hparams)
print(model.summary())

model_path = "SAVED_MODELS/{ts}".format(ts=datetime.now()).replace(" ", "-").replace(":", "-").split(".")[0]
os.mkdir(model_path)

checkpoint_path = "{modelPath}/{mn}.ckpt".format(modelPath=model_path, mn=model.name)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

EPOCHS = 10
BATCH_SIZE = 4

#train
train_hist = model.fit(
    x=x_train,
    y=y_train,
    epochs=EPOCHS,
    validation_data=(x_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[cp_callback],
    shuffle=True
)

#write to training log
log_entry = [
    dataset_name,
    model.name,
    model_path,
    SEQ_LEN,
    EPOCHS,
    BATCH_SIZE,
    train_hist.history["val_mean_absolute_error"][-1]
]

with open('SAVED_MODELS/training_log.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(log_entry)

Model: "LSTNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 attention_2 (Attention)     multiple                  0         
                                                                 
 attention_3 (Attention)     multiple                  0         
                                                                 
 simpleLSTM (SimpleLSTM)     multiple                  6353      
                                                                 
Total params: 6,353
Trainable params: 6,353
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
7092/7092 [==============================] - ETA: 0s - loss: 10.9753 - mean_absolute_error: 1.3997
Epoch 1: saving model to SAVED_MODELS/2023-11-12-19-07-03\LSTNet.ckpt
7092/7092 [==============================] - 1095s 154ms/step - loss: 10.9753 - mean_absolute_error: 1.3997 - val_loss: 381.0732 - val_m

KeyboardInterrupt: 

## Testing

The below code is for testing and visualization.

In [ ]:
#make sequences, x/y split, normalize, and fold
""" tdx, tdy = DataProcessor.sequence_target_split(DataProcessor.sliding_window_sequence(val_data, SEQ_LEN), target_size=1)
tdx = DataProcessor.fold_sequences(DataProcessor.zscore_standardization(tdx))
tdy = DataProcessor.fold_sequences(tdy) """

tdx, tdy = DataProcessor.sequence_target_split(DataProcessor.sliding_window_sequence(test_data, SEQ_LEN), target_size=1)
tdx = DataProcessor.zscore_standardization(tdx)

tdx = DataProcessor.fold_sequences(tdx)
tdy = DataProcessor.fold_sequences(tdy)

if len(tdy.shape) == 1:
    print("rrr")

    tdx = tdx.reshape((tdx.shape[0], tdx.shape[1], 1))
    tdy = tdy.reshape((tdy.shape[0], 1))

tdy = tdy[:,Y_VAR]

# evaluate model on the test set
print(model.evaluate(tdx, tdy, batch_size=BATCH_SIZE))

plt.plot(np.arange(0, tdy.shape[0]), tdy, 'green')   #actual data
plt.plot(np.arange(0, tdx.shape[0]), model.predict(tdx).reshape(tdx.shape[0]), 'blue')    #predicted data
plt.title(model_path)

plt.savefig('SAVED_MODELS/model_test_visualizations/{mi}.png'.format(mi=model.name + "_" + model_path.split("/")[1]))
plt.show()